In [1]:
from url import *
from rapidfuzz import process

In [2]:
url = "https://www.allrecipes.com/mediterranean-baked-cod-with-lemon-recipe-8576313"
# fetch_page_from_url(url)

### Test extract methods per step below ###

ingredients, instructions = fetch_page_from_url(url)

print(instructions)
print('\n')

vegan_transform = transform_recipe(ingredients, instructions, to_vegetarian)
print(vegan_transform['ingredients'])
print(vegan_transform['instructions'])
print('\n')

# healthy_transform = transform_recipe(ingredients, instructions, to_healthy)
# print(healthy_transform['ingredients'])
# print(healthy_transform['instructions'])
# print('\n')

# mexican_transform = transform_recipe(ingredients, instructions, mexican_style['ingredients'], mexican_style['techniques'])
# print(mexican_transform['ingredients'])
# print(mexican_transform['instructions'])
# print('\n')

# italian_transform = transform_recipe(ingredients, instructions, italian_style['ingredients'], italian_style['techniques'])
# print(italian_transform['ingredients'])
# print(italian_transform['instructions'])
# print('\n')

['Preheat the oven to 400 degrees F (200 degrees C).', 'Place softened butter, minced garlic, parsley, oregano, and thyme or rosemary on a cutting board', 'Using a sharp knife, cut herbs and garlic into each other and the butter, cutting and mixing as you go', 'Add pink salt, black pepper, and paprika, and mix until well blended.', 'Pat cod filets dry', 'In a 12x18-inch casserole or baking pan, place each filet on top of 2 lemon slices', 'Evenly divide herb butter mixture among the filets; use a fork or offset spatula to spread herb butter over filets', 'Top each filet with 2 remaining lemon slices.', 'Bake in the preheated oven until cod flakes easily with a fork, 13 to 15 minutes', 'See note.', 'To serve, drizzle each filet with extra virgin olive oil, and garnish with fresh parsley, if desired.']


[{'quantity': '3', 'unit': 'tablespoons', 'name': ' butter', 'descriptor': 'unsalted', 'preparation': 'softened'}, {'quantity': '1', 'unit': 'tablespoon', 'name': ' garlic', 'preparation'

In [9]:
line = "Pat cod filets dry"
for old_ing, new_ing in to_vegetarian.items():
    if old_ing in line.lower():
        line = line.replace(old_ing, ' or '.join(new_ing))
line

'Pat tofu or seaweed-marinated tofu filets dry'

In [8]:
from pyfood.utils import Shelf
shelf = Shelf()

In [9]:
for i in ingredients:
    print(i)
    print(shelf.process_ingredients([i['name'] + i['descriptor'] if 'discriptor' in i.keys() else i['name']]))

{'quantity': '3', 'unit': 'tablespoons', 'name': ' butter', 'descriptor': 'unsalted', 'preparation': 'softened'}
{'ingredients': [{'foodname': 'butter', 'quantity': 100.0, 'unit': 'g', 'taxon': '201'}], 'HS': [], 'ingredients_by_taxon': [['butter']], 'labels': {'vegan': False, 'vege': True, 'seasonality': {'city': 'EU', 'score': 50, 'best_now': True}}, 'revisited': []}
{'quantity': '1', 'unit': 'tablespoon', 'name': ' garlic', 'preparation': 'finely minced', 'descriptor': 'fresh'}
{'ingredients': [{'foodname': 'garlic', 'quantity': 100.0, 'unit': 'g', 'taxon': '002'}], 'HS': [], 'ingredients_by_taxon': [['garlic']], 'labels': {'vegan': True, 'vege': True, 'seasonality': {'city': 'EU', 'score': 100, 'best_now': True}}, 'revisited': []}
{'quantity': '1', 'unit': 'tablespoon', 'name': ' parsley', 'preparation': 'minced', 'descriptor': 'fresh'}
{'ingredients': [{'foodname': 'parsley', 'quantity': 100.0, 'unit': 'g', 'taxon': '008'}], 'HS': [], 'ingredients_by_taxon': [['parsley']], 'labels

NotFittedError: This TfidfTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [10]:
shelf.process_ingredients(['cod filet'])

NotFittedError: This TfidfTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.